In [1]:
import pandas as pd
import csv
import time
import math
import re

In [2]:
def replace_chars(row, column_name):
    return re.sub('[,;]', ' ', row[column_name])

In [3]:
def rearrange_csv_columns(path):
    new_path = '/'.join(path.split('/')[:-1]) + '/new_' + path.split('/')[-1]
    num_of_cols = 35
    prob_col =  11
    prob_string_list = [' Jr."' , ' Jr.']
     
    with open(path, "r") as f_in, open(new_path, "w") as f_out:
        reader = csv.reader(f_in)
        writer = csv.writer(f_out)
        for row in reader:
            if len(row) == num_of_cols + 1:
                assert row[prob_col] in prob_string_list, row[prob_col] + ' -> id:' + str(row[0])
                new_row[:prob_col-1] = row[:prob_col-1]
                new_row[prob_col-1] = row[prob_col-1] + row[prob_col]
                new_row[prob_col:] = row[prob_col+1:]
            else:
                new_row = row

            writer.writerow(new_row)

    f_in.close()
    f_out.close()
    
    return new_path

In [4]:
# subjects 11, 22 are missing => 21 subjects in total
subjects_dict = dict()
with open('../data/policy_agenda.csv', "r") as f_in:
    reader = csv.reader(f_in)
    for row in reader:
        subjects_dict[float(row[0])] = row[1]
    f_in.close()

##### inner bills file

In [5]:
# _ = rearrange_csv_columns('data/bills.csv')
bills_initial = pd.read_csv('../data/new_bills.csv', low_memory=False)

In [6]:
bills_initial = bills_initial[ bills_initial['isBill'] == 1]

In [7]:
print(bills_initial.shape)
print(bills_initial.columns)
print(bills_initial.Cong.unique())
print(bills_initial.Major.unique())

(236317, 35)
Index(['idNEW', 'idOLD', 'BillNum', 'BillType', 'Cong', 'IntrDate',
       'ShortTitle', 'OfficialTitle', 'PopTitle', 'SpThomasID', 'SpName',
       'SpState', 'SpDist', 'UpdatedAt', 'CoSpThID', 'PLawNo', 'MinorBill',
       'PLawNoFull', 'compLaw', 'Major', 'Minor', 'ChRef', 'RankRef', 'MemRef',
       'SubChRef', 'SubRankRef', 'ImpBill', 'URL', 'isBill', 'Majority',
       'Senate', 'LeadRef', 'SpParty', 'commRefs', 'timestamp'],
      dtype='object')
[ 93  94  95  96  97  98  99 100 101 102 103 104 105 106 107 108 109 110
 111 112 113 114]
[20.  6.  4. 16.  1. 14.  7.  5. 17. 13. 10. 21.  2. 15.  3. 12. 99.  8.
 18. 19.  9. nan]


In [8]:
BILL_COLUMNS = ['Cong', 'SpParty', 'Major', 'SpState', 'idNEW', 'URL', 'OfficialTitle']
# bills_initial = bills_initial[BILL_COLUMNS]

In [9]:
bills = pd.DataFrame(index=bills_initial.index)
bills[BILL_COLUMNS] = bills_initial[BILL_COLUMNS]
bills['Major'] = bills_initial['Major'].map(subjects_dict) 
bills['Major'].fillna('Other', inplace=True)

In [10]:
print(bills.shape)
print(bills.columns)
bills.head(1)

(236317, 7)
Index(['Cong', 'SpParty', 'Major', 'SpState', 'idNEW', 'URL', 'OfficialTitle'], dtype='object')


,Cong,SpParty,Major,SpState,idNEW,URL,OfficialTitle
0,93,D,Government Operations,IL,1,http://beta.congress.gov/bill/93th-congress/ho...,A bill to provide for the establishment of an ...


### Actions

In [11]:
actions = pd.read_csv('../data/actions.csv', low_memory=False)

print(actions.shape)
print(actions.columns)
print(actions.status.unique())

(828172, 14)
Index(['billID', 'bill', 'cong', 'billtype', 'acted_at', 'loc', 'status',
       'actno', 'subbill', 'phrase', 'manUpdate', 'goodDate', 'timestamp',
       'actionID'],
      dtype='object')
['INT' 'COMM' 'REP' 'PASSO' 'PASSB' 'WAIT' 'SIGN' 'LAW' 'PASS' 'DISC'
 'CREP' 'VETO' 'RESA' 'RULP' 'CAL' 'PASSC' 'CONF' 'PASSV2' 'CONS' 'PASSV']


In [12]:
bills = pd.merge(bills, actions[['status']], left_index=True, right_index=True)

In [13]:
print(bills.shape)
print(bills.columns)
bills.head(1)

(236317, 8)
Index(['Cong', 'SpParty', 'Major', 'SpState', 'idNEW', 'URL', 'OfficialTitle',
       'status'],
      dtype='object')


,Cong,SpParty,Major,SpState,idNEW,URL,OfficialTitle,status
0,93,D,Government Operations,IL,1,http://beta.congress.gov/bill/93th-congress/ho...,A bill to provide for the establishment of an ...,INT


In [14]:
def define_status(row):
    if row['status'] == 'LAW' :
        return 'passed'
    else:
        return 'not_passed'

In [15]:
bills['status'] = bills.apply(lambda row: define_status(row),axis=1)

In [16]:
bills['status'].unique()

array(['not_passed', 'passed'], dtype=object)

In [17]:
bills.rename(columns={'Cong':'congress', 'SpParty':'party', 'Major':'major', 'SpState':'state', 
                      'URL':'url', 'OfficialTitle':'title'}, inplace=True)

In [18]:
print(bills.shape)
print(bills.columns)
bills.head(1)

(236317, 8)
Index(['congress', 'party', 'major', 'state', 'idNEW', 'url', 'title',
       'status'],
      dtype='object')


,congress,party,major,state,idNEW,url,title,status
0,93,D,Government Operations,IL,1,http://beta.congress.gov/bill/93th-congress/ho...,A bill to provide for the establishment of an ...,not_passed


In [19]:
# BILL_COLUMNS = ['Cong', 'SpParty', 'Major', 'SpState', 'idNEW', 'URL', 'OfficialTitle', 'status']
# # bills['status'] = bills['is_bill']
# bills = bills[BILL_COLUMNS]
# # bills.columns = ['congress', 'party', 'major', 'state', 'idNEW', 'url', 'title', 'status']

### Congress 115

In [20]:
bills_115_intro = pd.read_csv('../data/bills_115.csv', low_memory=False)
bills_115_intro['status'] = 'not_passed'
bills_115_passed = pd.read_csv('../data/bills_115_passed.csv', low_memory=False)
bills_115_passed['status'] = 'passed'

In [21]:
print(bills_115_intro.shape, bills_115_passed.shape)
set.intersection(set(bills_115_intro.bill_id.values), set(bills_115_passed))

(4574, 7) (531, 7)


set()

In [22]:
bills_115 = pd.concat([bills_115_intro, bills_115_passed], ignore_index=True)
bills_115['congress'] = 115
# bills_115.columns = ['idNEW', 'party', 'major', 'state', 'url', 'title', 'congress', 'status']

In [23]:
bills_115.rename(columns={'bill_id':'idNEW', 'sponsor_party':'party', 'primary_subject':'major', 'sponsor_state':'state', 
                      'congressdotgov_url':'url'}, inplace=True)
bills_115 = bills_115[['congress', 'party', 'major', 'state', 'idNEW', 'url', 'title', 'status']]

In [24]:
transformation_dict = dict()
with open('../data/major_matching.txt', "r") as f_in:
    lines = f_in.readlines()
    f_in.close()

for line in lines:
    entry = line.split(';')
    transformation_dict[entry[0]] = subjects_dict[
        float(entry[1].replace('\n', ''))]
    
bills_115['major'] = bills_115['major'].map(transformation_dict) 
bills_115['major'].fillna('Other', inplace=True)

In [25]:
print(bills_115.shape)
bills_115.head(1)

(5105, 8)


,congress,party,major,state,idNEW,url,title,status
0,115,D,Other,CA,sres717-115,https://www.congress.gov/bill/115th-congress/s...,A resolution honoring the life and legacy of R...,not_passed


In [26]:
bills_115.status.value_counts().to_dict()

{'not_passed': 4574, 'passed': 531}

In [27]:
# test = pd.merge(bills, bills_115,  how='outer', left_on='idNEW', right_on = 'idNEW')
bills = pd.concat([bills, bills_115], ignore_index=True)
print(bills.shape)
bills.head(1)

(241422, 8)


,congress,party,major,state,idNEW,url,title,status
0,93,D,Government Operations,IL,1,http://beta.congress.gov/bill/93th-congress/ho...,A bill to provide for the establishment of an ...,not_passed


In [28]:
bills['title'] = bills.apply(lambda row: replace_chars(row, 'title'),axis=1)

### Grouping

In [29]:
# Before:
bills_count = bills.groupby(['congress', 'party', 'major', 'state', 'status']).size()
grouped_bills_count = bills_count.to_frame()
grouped_bills_count.columns = ['count']
grouped_bills_count.head(5)

# # Testing:
# bills_count = bills.groupby(['congress', 'party', 'major', 'state', 'status'], as_index=False)['idNEW'].agg('count')
# # bills_count[bills_count['congress']==115]['idNEW'].sum()

# bills_count.set_index(['congress', 'party', 'major', 'state', 'status'], inplace=True)
# bills_count.rename(columns={'idNEW':'count'}, inplace=True)
# bills_count.iloc[grouped_bills_count.index.get_level_values('congress') == 115]['count'].sum()# bills_count.head()

count
congress party major                state status           
93       D     Agriculture and Food AL    not_passed      1
                                    AR    not_passed     17
                                          passed          1
                                    AZ    not_passed      1
                                    CA    not_passed     29

In [30]:
grouped_bills_count.iloc[
    grouped_bills_count.index.get_level_values('congress') == 115]['count'].sum()# bills_count.head()

5105

In [31]:
grouped_id_list = bills.groupby(['congress', 'party', 'major', 'state', 'status'], as_index=False)['idNEW'].agg(
    {'id_list':(lambda x: list(x))})
grouped_id_list.set_index(['congress', 'party', 'major', 'state', 'status'], inplace=True)
grouped_id_list.head(5)

id_list
congress party major                state status                                                       
93       D     Agriculture and Food AL    not_passed                                            [24509]
                                    AR    not_passed  [1614, 2152, 3042, 5288, 5571, 7787, 8518, 106...
                                          passed                                                [22443]
                                    AZ    not_passed                                            [12457]
                                    CA    not_passed  [646, 652, 690, 1862, 1877, 2804, 2863, 3029, ...

In [32]:
grouped_url_list = bills.groupby(['congress', 'party', 'major', 'state', 'status'], as_index=False)['url'].agg(
    {'url_list':(lambda x: list(x))})
grouped_url_list.set_index(['congress', 'party', 'major', 'state', 'status'], inplace=True)
grouped_url_list.head(5)

url_list
congress party major                state status                                                       
93       D     Agriculture and Food AL    not_passed  [http://beta.congress.gov/bill/93th-congress/s...
                                    AR    not_passed  [http://beta.congress.gov/bill/93th-congress/h...
                                          passed      [http://beta.congress.gov/bill/93th-congress/s...
                                    AZ    not_passed  [http://beta.congress.gov/bill/93th-congress/h...
                                    CA    not_passed  [http://beta.congress.gov/bill/93th-congress/h...

In [33]:
grouped_title_list = bills.groupby(['congress', 'party', 'major', 'state', 'status'], as_index=False)['title'].agg(
    {'title_list':(lambda x: list(x))})
grouped_title_list.set_index(['congress', 'party', 'major', 'state', 'status'], inplace=True)
grouped_title_list.head(5)

title_list
congress party major                state status                                                       
93       D     Agriculture and Food AL    not_passed  [A bill to amend the Egg Products Inspection A...
                                    AR    not_passed  [A bill to amend the emergency loan program un...
                                          passed      [A bill authorizing the Secretary of Agricultu...
                                    AZ    not_passed  [A bill to prevent the estate tax law from ope...
                                    CA    not_passed  [A bill to amend the Federal Meat Inspection A...

In [34]:
grouped_bills = pd.merge(grouped_id_list, grouped_url_list, left_index=True, right_index=True)
grouped_bills = pd.merge(grouped_bills, grouped_title_list, left_index=True, right_index=True)
grouped_bills = pd.merge(grouped_bills, grouped_bills_count, left_index=True, right_index=True)
grouped_bills.head(5)

id_list  \
congress party major                state status                                                          
93       D     Agriculture and Food AL    not_passed                                            [24509]   
                                    AR    not_passed  [1614, 2152, 3042, 5288, 5571, 7787, 8518, 106...   
                                          passed                                                [22443]   
                                    AZ    not_passed                                            [12457]   
                                    CA    not_passed  [646, 652, 690, 1862, 1877, 2804, 2863, 3029, ...   

                                                                                               url_list  \
congress party major                state status                                                          
93       D     Agriculture and Food AL    not_passed  [http://beta.congress.gov/bill/93th-congress/s...   
                                    AR    not_passed  [http://beta.congress.gov/bill/93th-congress/h...   
                                          passed      [http://beta.congress.gov/bill/93th-congress/s...   
                                    AZ    not_passed  [http://beta.congress.gov/bill/93th-congress/h...   
                                    CA    not_passed  [http://beta.congress.gov/bill/93th-congress/h...   

                                                                                             title_list  \
congress party major                state status                                                          
93       D     Agriculture and Food AL    not_passed  [A bill to amend the Egg Products Inspection A...   
                                    AR    not_passed  [A bill to amend the emergency loan program un...   
                                          passed      [A bill authorizing the Secretary of Agricultu...   
                                    AZ    not_passed  [A bill to prevent the estate tax law from ope...   
                                    CA    not_passed  [A bill to amend the Federal Meat Inspection A...   

                                                      count  
congress party major                state status             
93       D     Agriculture and Food AL    not_passed      1  
                                    AR    not_passed     17  
                                          passed          1  
                                    AZ    not_passed      1  
                                    CA    not_passed     29

In [35]:
final_grouped = pd.DataFrame(index=grouped_bills.index) 
final_grouped['id_list'] = [ ';'.join([str(x) for x in row]) for row in grouped_bills['id_list']]
final_grouped['url_list'] = [ ';'.join([str(x) for x in row]) for row in grouped_bills['url_list']]
final_grouped['title_list'] = [ ';'.join([str(x) for x in row]) for row in grouped_bills['title_list']]
final_grouped['count'] = grouped_bills_count['count']
# final_grouped['congress'] = final_grouped['Cong']
# final_grouped['party'] = final_grouped['SpParty']
# final_grouped['major'] = final_grouped['Major']
# final_grouped['state'] = final_grouped['SpState']
# final_grouped = final_grouped[['congress','party','major','state','id_list','url_list','count']]
# final_grouped.index.names = ['congress', 'party', 'major', 'state', 'status']
final_grouped.head()

id_list  \
congress party major                state status                                                          
93       D     Agriculture and Food AL    not_passed                                              24509   
                                    AR    not_passed  1614;2152;3042;5288;5571;7787;8518;10697;10903...   
                                          passed                                                  22443   
                                    AZ    not_passed                                              12457   
                                    CA    not_passed  646;652;690;1862;1877;2804;2863;3029;4514;5686...   

                                                                                               url_list  \
congress party major                state status                                                          
93       D     Agriculture and Food AL    not_passed  http://beta.congress.gov/bill/93th-congress/se...   
                                    AR    not_passed  http://beta.congress.gov/bill/93th-congress/ho...   
                                          passed      http://beta.congress.gov/bill/93th-congress/se...   
                                    AZ    not_passed  http://beta.congress.gov/bill/93th-congress/ho...   
                                    CA    not_passed  http://beta.congress.gov/bill/93th-congress/ho...   

                                                                                             title_list  \
congress party major                state status                                                          
93       D     Agriculture and Food AL    not_passed   A bill to amend the Egg Products Inspection Act.   
                                    AR    not_passed  A bill to amend the emergency loan program und...   
                                          passed      A bill authorizing the Secretary of Agricultur...   
                                    AZ    not_passed  A bill to prevent the estate tax law from oper...   
                                    CA    not_passed  A bill to amend the Federal Meat Inspection Ac...   

                                                      count  
congress party major                state status             
93       D     Agriculture and Food AL    not_passed      1  
                                    AR    not_passed     17  
                                          passed          1  
                                    AZ    not_passed      1  
                                    CA    not_passed     29

In [36]:
final_grouped.to_csv('../project/app/data/grouped_bills.csv', sep=',', encoding='utf-8', escapechar=';')

In [37]:
total_ids = 0
total_titles = 0
total_urls = 0

for index, row in final_grouped.iloc[final_grouped.index.get_level_values('congress')==115].iterrows():
    ids = row['id_list'].split(';')
    titles = row['title_list'].split(';')
    urls = row['url_list'].split(';')
    
    total_ids += len(ids)
    total_titles += len(titles)
    total_urls += len(urls)
    

In [38]:
print(total_ids, total_titles, total_urls)

5105 5105 5105


In [39]:
final_grouped.iloc[final_grouped.index.get_level_values('congress')==115]

id_list  \
congress party major                state status                                                          
115      D     Agriculture and Food AL    not_passed                                          s3117-115   
                                    CA    not_passed                sres540-115;sres248-115;sres208-115   
                                    CO    not_passed                      s2997-115;s2989-115;s1576-115   
                                    CT    not_passed            s2813-115;s2477-115;s1805-115;s1680-115   
                                    DE    not_passed                                        sres679-115   
                                    HI    not_passed                                 s2479-115;s541-115   
                                    IL    not_passed                                        sres271-115   
                                    IN    not_passed                      s2263-115;s2137-115;s1678-115   
                                    MD    not_passed                                s2839-115;s2139-115   
                                    MI    not_passed            s3005-115;s2822-115;s2552-115;s1966-115   
                                    MN    not_passed  s3062-115;s2874-115;s2619-115;s2487-115;sres40...   
                                    MT    not_passed                                          s3104-115   
                                    ND    not_passed            s3407-115;s2762-115;s2190-115;s1998-115   
                                    NH    not_passed                      s2790-115;s2713-115;s2086-115   
                                    NJ    not_passed                                s2624-115;s1706-115   
                                    NM    not_passed            s2685-115;s2409-115;s2133-115;s1064-115   
                                    NY    not_passed  s3034-115;s2723-115;s2555-115;s2426-115;s1707-...   
                                    OH    not_passed                                s2749-115;s1947-115   
                                    OR    not_passed                            sres532-115;sres189-115   
                                          passed                                            sres189-115   
                                    PA    not_passed  s2978-115;s2824-115;s2404-115;s2215-115;s2085-115   
                                    VA    not_passed                                s2733-115;s2569-115   
                                    VT    not_passed                                          s1767-115   
                                    WA    not_passed                                          s3268-115   
                                    WI    not_passed            s3113-115;s2964-115;s2927-115;s2712-115   
               Civil Rights         AL    not_passed                                          s3191-115   
                                    CA    not_passed  s2918-115;sres448-115;s1114-115;sres147-115;sr...   
                                    DC    passed                                             hr2989-115   
                                    DE    not_passed                                          s2186-115   
                                    HI    not_passed  s3110-115;sres546-115;sres530-115;sres387-115;...   
...                                                                                                 ...   
         R     Transportation       IN    not_passed                      s3643-115;s3402-115;s3352-115   
                                    KY    not_passed                                 s1298-115;s531-115   
                                          passed                                              hr302-115   
                                    LA    not_passed                                          s2556-115   
                                    ME    not_passed                              sres491-115;s2244-115   
                              